In [7]:
#Directions

#make sure you have installed cv2 with command: 
#  pip install opencv-python
#download the file called pose_iter_584000.caffemodel which is saved in this dropbox:
#  https://www.dropbox.com/s/3x0xambj2rkyrap/pose_iter_584000.caffemodel?dl=0
#download the file called pose_deploy.prototxt which is saved here:
#  https://github.com/CMU-Perceptual-Computing-Lab/openpose/tree/master/models/pose/body_25

#save both of these files in the same folder as this ipynb
#save a video in the same folder or subfolder

#issues:

# the model keeps recognizing exercise equipment in the background as body parts,
# i think using a different model(for multiple people simultaneously), 
# or adjusting confidence could help with this

# i could only make the scaling work by cropping the images to squares first
# not a big deal now with our training data, but it could cause problems in the future

# i think most of these .avi videos are 30 fps. if not, framestep() will need to be updated

# this script does not save the video, it only displays it in a popup window. 


In [8]:
import cv2

import csv

# For GPU server
#path = '/lfs/mead9103.ui/NN_Training/'
path = ''

def video_to_CSV(filename, frames = 240):
    # Open the video file
    cap = cv2.VideoCapture(filename)

    if not cap.isOpened():
        print("Cannot open camera for video %s" % filename)
        return

    # Define the keypoint mapping for this OpenPose body_25 model
    keypoints_mapping = {
        0:  "Nose", 1:  "Neck", 2:  "RShoulder", 3:  "RElbow", 4:  "RWrist", 5:  "LShoulder", 6:  "LElbow",
        7:  "LWrist", 8:  "MidHip", 9:  "RHip", 10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee",
        14: "LAnkle", 15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe", 20: "LSmallToe",
        21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel"
    }
    #load the model
    net = cv2.dnn.readNetFromCaffe(path + 'pose_deploy.prototxt', path + 'pose_iter_584000.caffemodel')

    #Write frame information of joint locations into a csv file
    f = open(filename + ".csv", 'w', newline='')
    w = csv.writer(f)
    #for keypoint, label in keypoints_mapping:
        #w.write(str(keypoints_mapping.items())
    #w.writerow(keypoints_mapping.values())
    
    def framestep(cap):
        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        new_vid = []
        # Keep stepsize as a float, consider someday skipping frame 0 since it's unlikely to be useful
        stepsize = (frame_count - 1) / (frames - 1) # subtract 1 to account for frame 0
        print("Video: %s, frames to be analyzed: %d (%d / %.2f)" % (filename, frames, frame_count, stepsize))
        curr_frame = 0
        i = 0 # float index of next frame
        
        # Loop until the whole video has been read, or the requested number of frames are reached
        while curr_frame < frame_count:
            ret, frame = cap.read()
            if curr_frame == int(i):
                print("Starting frame: %d (index %.2f)" % (curr_frame, i))
                new_vid.append(pose(frame))
                i += stepsize
            curr_frame += 1
        return new_vid
        
    def squarify(frame):
        height, width, _ = frame.shape
        min_dim = min(height, width)

        # Calculate the cropping dimensions
        crop_height = (height - min_dim) // 2
        crop_width = (width - min_dim) // 2

        # Crop the image equally from both sides to make it a square
        return frame[crop_height:crop_height+min_dim, crop_width:crop_width+min_dim] , min_dim

    def pose(frame):
        frame, size = squarify(frame)
        blob = cv2.dnn.blobFromImage(frame, 1/255, (size, size),
                                (0, 0, 0), swapRB=False, crop=True)

        # run forward pass to get the pose estimation
        net.setInput(blob)
        output = net.forward()
        
        # Extract joint locations
        joint_locations = []
        csv_joint_locations = []

        for i in range(len(keypoints_mapping)): #-1 bc we dont want point for the background
            keypoint = output[0, i, :, :]
            min_val, confidence, min_loc, point = cv2.minMaxLoc(keypoint)

            #if confidence > 0.1:  # can adjust the confidence threshold if needed ???
            joint_locations.append((8 * int(point[0]), 8 * int(point[1]), 0)) #for testing/human readable
            csv_joint_locations.append(8 * int(point[0])) #to be printed into csv
            csv_joint_locations.append(8 * int(point[1])) #to be printed into csv
            csv_joint_locations.append(0) #to be printed into csv
            '''else:
                joint_locations.append(None) #for testing/human readable
                csv_joint_locations.append(8 * int(point[0])) #to be printed into csv
                csv_joint_locations.append(8 * int(point[1])) #to be printed into csv
            '''
        #joint_locations contains the locations of the detected joints and corresponding index

        '''for location in joint_locations:
            if location:
                x, y, index = location
                cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
                #cv2.putText(image, str(index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        '''        
        #print joint locations into a csv file
        w.writerow(csv_joint_locations)
        
        return frame

    #write the information + circles onto each frame and pop up a window for each frame
    my_video = framestep(cap)
    size = my_video[0].shape[1], my_video[0].shape[0]
    print(size)
    '''out = cv2.VideoWriter("real.avi", cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

    for i in range(len(my_video)):
        out.write(my_video[i])
        cv2.imshow("video", my_video[i])
        cv2.waitKey(0)

    cv2.destroyAllWindows()
    out.release()
    '''
    cap.release()
    f.close()

In [9]:
#to write the frames into a video format for viewer presentation
'''
cap = cv2.VideoCapture(video_path)

def test():
    size = (1080, 1080)
    out = cv2.VideoWriter("deep.avi", cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
    for i in range(0, 200):
        f, s = squarify(cap.read()[1])
        s
        out.write(f)
    
    out.release()

test()
cap.release()
'''

'\ncap = cv2.VideoCapture(video_path)\n\ndef test():\n    size = (1080, 1080)\n    out = cv2.VideoWriter("deep.avi", cv2.VideoWriter_fourcc(*\'DIVX\'), 30, size)\n    for i in range(0, 200):\n        f, s = squarify(cap.read()[1])\n        s\n        out.write(f)\n    \n    out.release()\n\ntest()\ncap.release()\n'

In [10]:
#split the movements in the file into 10 reps (240 frames/arrays)
#estimating by dividing by 10 for now to seperate the reps
#future me note: can divide each rep by the local minimum of the hip value within x number of frames to obtain the full repinfo

import numpy as np

'''
Reads the joint location data from the provided csv file, assuming that the columns are joint positions, and
the rows are individual frames. A numpy array is returned with the columns representing frames per episode,
and the rows representing joint positions * episodes * subjects.
'''
def episode_split(filename, frames):
    #load file into a single array (all 240 arrays into 1)
    with open(filename, 'r', newline='') as f:
        w = csv.reader(f, delimiter = ',', quoting = csv.QUOTE_NONE)

        raw_data = np.zeros(shape = (frames, 75))

        # Populate raw_data with each row representing 1 frame
        i = 0
        for row in w:
            if i >= frames:
                break

            raw_data[i, :] = row
            #print("i: ", i, " row: ", row)
            i += 1

        #print(raw_data.shape)

        raw_data_1 = np.reshape(raw_data, 75 * frames)
        #print(raw_data_1)
        raw_data_2 = raw_data_1.reshape(75, frames, order = 'F')
        #print(raw_data_2)

        # Split data into 10 episodes, with an equal number of frames per episode and the last episode getting the remainder

        # Number of frames to keep in each episode, rounded down
        n = int(frames / 10) # nframes/10

        split_data = np.zeros(shape = (75 * 10, n)) # list of 10 lists, first 9 with n frames of joint locations, 10th with all the rest
        for i in range(0, 9):
            split_data[i * 75:(i + 1) * 75, :] = raw_data_2[:, i * n:(i + 1) * n]

        split_data[9 * 75:, :] = raw_data_2[:, 9 * n:]
        #split_data.append(raw_data_2[9 * n:len(xs)]) # put the rest of the frames into episode 10

        #print(len(split_data))
        #print(split_data)
        f.close()
        return np.array(split_data)

#print(episode_split("Videos/subject001/DeepSquat1.avi.csv", 10))


In [35]:
#Given episdode split/smoothed data into model
#append correct movements into CSV
#append incorrect movements into CSV
#add z point to every point
#take in movement and subject to generate filenames
def combineEpisodes(mov, sub, cor):
    frames = 40
    ceps = np.zeros(shape = (75 * 10, frames)) # all episodes for this subject+movement combined
    
    # Loop through all episodes
    for i in range(0, 10):
        with open("Smoothed_Data_CSVs/" + ("C" if cor else "I") + "_m0" + str(mov) + "_s" + ("0" if sub < 10 else "") + str(sub) + "_e" + str(i) + ".csv",  'r', newline='') as f:
            w = csv.reader(f, delimiter = ',', quoting = csv.QUOTE_NONE)

            raw_data = np.zeros(shape = (frames, 50))

            # Populate raw_data with each row representing 1 frame
            j = 0
            for row in w:
                if j >= frames:
                    break

                raw_data[j, :] = row
                #print("i: ", i, " row: ", row)
                j += 1
                
            #insert zeros for z points for every joint (every three columns for 25 joints)
            j = range(2, 51, 2)
            k = 0 #[0] * frames
            raw_data = np.insert(raw_data, j, k, axis = 1)
         
            raw_data_1 = raw_data.reshape(75 * frames)
            raw_data_2 = raw_data_1.reshape(75, frames, order = 'F')
            
            ceps[75 * i:75 * (i + 1), :] = raw_data_2
            f.close()
    return ceps

In [36]:
episodes = 10
features = 75

'''video_to_CSV("Molly_Correct_DeepSquat.mp4") #get the data for correct squat
video_to_CSV("Molly_Incorrect_DeepSquat.mp4") #get the data for incorrect squat
'''
subjects = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11] # there is no subject 2
combined = np.zeros(shape = (len(subjects) * episodes * features, 400//10))
ind = 0
for i in subjects:
    combined[ind * episodes * features:(ind + 1) * episodes * features, :] = combineEpisodes(1, i, True)
    ind += 1

combined_inc = np.zeros(shape = (len(subjects) * episodes * features, 400//10))
ind = 0
for i in subjects:
    combined_inc[ind * episodes * features:(ind + 1) * episodes * features, :] = combineEpisodes(1,i,False)
    ind += 1  

#get the mean of every first value
data_mean = np.mean(combined, axis = 0)

#inc_data
data_mean_inc = np.mean(combined_inc, axis = 0)

centered_data = combined - data_mean

#inc_data
centered_data_inc = combined_inc - data_mean_inc

# Scale the data between -1 and 1
scaling_value = np.ceil(max(np.max(centered_data), abs(np.min(centered_data))))
data_correct = centered_data / scaling_value

# Scale the incorrect data between -1 and 1
scaling_value_inc = np.ceil(max(np.max(centered_data_inc), abs(np.min(centered_data_inc))))
data_incorrect = centered_data_inc / scaling_value_inc

In [15]:
'''# The shortest video has 417 frames
frames_to_analyze = 3
episodes = 10
features = 75

subjects = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11] # there is no subject 2

#correct data
combined = np.zeros(shape = (len(subjects) * episodes * features, frames_to_analyze//10))
ind = 0
for i in subjects:
    combined[ind * episodes * features:(ind + 1) * episodes * features, :] = episode_split(path + "Videos/subject0" + ("0" if i < 10 else '') + str(i)  + "/DeepSquat1.avi.csv", frames_to_analyze)
    ind += 1
    
#incorrect data
combined_inc = np.zeros(shape = (len(subjects) * episodes * features, frames_to_analyze//10))
ind = 0
for i in subjects:
    combined_inc[ind * episodes * features:(ind + 1) * episodes * features, :] = episode_split(path + "Videos/subject0" + ("0" if i < 10 else '') + str(i)  + "/DeepSquat2.avi.csv", frames_to_analyze)
    ind += 1  

combined[combined == ''] = 0
combined = combined.astype(float)
#print(combined.shape)
#print(combined)

combined_inc[combined_inc == ''] = 0
combined_inc = combined_inc.astype(float)

#get the mean of every first value
data_mean = np.mean(combined, axis = 0)

#inc_data
data_mean_inc = np.mean(combined_inc, axis = 0)

centered_data = combined - data_mean

#inc_data
centered_data_inc = combined_inc - data_mean_inc

# Scale the data between -1 and 1
scaling_value = np.ceil(max(np.max(centered_data), abs(np.min(centered_data))))
data_correct = centered_data / scaling_value

# Scale the incorrect data between -1 and 1
scaling_value_inc = np.ceil(max(np.max(centered_data_inc), abs(np.min(centered_data_inc))))
data_incorrect = centered_data_inc / scaling_value_inc

#print(len(data_correct))
print(data_correct)
'''

'# The shortest video has 417 frames\nframes_to_analyze = 3\nepisodes = 10\nfeatures = 75\n\nsubjects = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11] # there is no subject 2\n\n#correct data\ncombined = np.zeros(shape = (len(subjects) * episodes * features, frames_to_analyze//10))\nind = 0\nfor i in subjects:\n    combined[ind * episodes * features:(ind + 1) * episodes * features, :] = episode_split(path + "Videos/subject0" + ("0" if i < 10 else \'\') + str(i)  + "/DeepSquat1.avi.csv", frames_to_analyze)\n    ind += 1\n    \n#incorrect data\ncombined_inc = np.zeros(shape = (len(subjects) * episodes * features, frames_to_analyze//10))\nind = 0\nfor i in subjects:\n    combined_inc[ind * episodes * features:(ind + 1) * episodes * features, :] = episode_split(path + "Videos/subject0" + ("0" if i < 10 else \'\') + str(i)  + "/DeepSquat2.avi.csv", frames_to_analyze)\n    ind += 1  \n\ncombined[combined == \'\'] = 0\ncombined = combined.astype(float)\n#print(combined.shape)\n#print(combined)\n\ncombined_

In [37]:
# Indices of episodes to keep
good_indices = [(2, 10), (12, 20), (22, 30), (32, 40), (42, 60), (62, 63), (65, 70), (72, 80), (82, 84), (86, 100)]

# Remove the first episodes for most of the subjects due to missing values during the data recording and also remove a few other inconsistent episodes
data_correct_red = np.zeros(shape = (90 * features, len(data_correct[0])));
i = 0 # number of episodes copied so far
for ind in good_indices:
    eps = ind[1] - ind[0] + 1 # number of episodes to copy for this range
    
    # Copy all of the features for every episode within this index's range
    data_correct_red[i * features:(i + eps) * features, :] = data_correct[(ind[0] - 1) * features:ind[1] * features, :]
    
    i += eps # record the number of episodes copied so far
    #print("episodes this time: ", eps, " total episodes so far: ", i)
    
#print(data_correct_red[6600:6750, :])


# Remove the first episodes for most of the subjects due to missing values during the data recording and also remove a few other inconsistent episodes
data_incorrect_red = np.zeros(shape = (90 * features, len(data_incorrect[0])));
i = 0 # number of episodes copied so far
for ind in good_indices:
    eps = ind[1] - ind[0] + 1 # number of episodes to copy for this range
    
    # Copy all of the features for every episode within this index's range
    data_incorrect_red[i * features:(i + eps) * features, :] = data_incorrect[(ind[0] - 1) * features:ind[1] * features, :]
    
    i += eps # record the number of episodes copied so far
    #print("episodes this time: ", eps, " total episodes so far: ", i)

In [38]:
f = open(path + 'smoothed_data_correct.csv', 'w', newline = '')
w = csv.writer(f)
np.apply_along_axis(w.writerow, axis = 1, arr = data_correct_red)
f.close()


f = open(path + 'smoothed_data_incorrect.csv', 'w', newline = '')
w = csv.writer(f)
np.apply_along_axis(w.writerow, axis = 1, arr = data_incorrect_red)

f.close()